Chat Message Memory

session id

In [2]:
# Lang Smith
from dotenv import load_dotenv

load_dotenv('./../.env')

True

In [3]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate)
from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(base_url=base_url, model=model)

template = ChatPromptTemplate.from_template("{prompt}")
chain = template | llm | StrOutputParser()

about = "My name is Sungjong Son. I work for DevCom"
chain.invoke({'prompt':about})

"Hello Sungjong Son, thank you for reaching out. It's great to connect with someone from the DevCom team. How can I assist you today?"

In [5]:
# Runnable with Message History (메시지 내용 기억시키기)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [8]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///chat_history.db")

In [9]:
runnable_with_history = RunnableWithMessageHistory(chain, get_session_history)

In [10]:
user_id = 'sungjong'
history = get_session_history(user_id)

history.get_messages()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1832\2599343957.py:2: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  history = get_session_history(user_id)


[]

In [20]:
history.clear()

In [21]:
runnable_with_history.invoke([HumanMessage(content=about)], config={'configurable': {'session_id':user_id}})

"Here's a more polished version:\n\n```\n[HumanMessage(\n    content='My name is Sungjong Son. I work for DevCom',\n    additional_kwargs={'context': 'devcom.com', 'language': 'eng'},\n    response_metadata={}\n)]\n```"

In [22]:
runnable_with_history.invoke([HumanMessage(content="What is my name?")], config={'configurable': {'session_id':user_id}})

'It seems like you\'re using the AIMessage class from the python-aiml library, which is used for building conversational interfaces. \n\nHere\'s an example of how you can use this class:\n\n```python\nfrom aiml import HumanMessage\n\ndef main():\n    # Define your message content\n    message = """\n    Hi, I\'m Sungjong Son.\n    My work is at DevCom.\n    """\n\n    # Create a HumanMessage object\n    human_message = HumanMessage(message)\n\n    # Print the response metadata\n    print("Response Metadata:")\n    for key, value in human_message.response_metadata.items():\n        print(f"{key}: {value}")\n\n    # Send the message and get the response\n    response = human_message.send()\n\n    # Parse the response\n    parsed_response = HumanMessage(parsed_response).parse_response()\n\n    # Print the response text\n    print("\\nResponse Text:")\n    for key, value in parsed_response.items():\n        print(f"{key}: {value}")\n\nif __name__ == "__main__":\n    main()\n```\n\nThis cod

In [23]:
# Message history with Dictionary Like Inputs
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate, MessagesPlaceholder)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system = SystemMessagePromptTemplate.from_template("You are helpful assistant.")
human = HumanMessagePromptTemplate.from_template("{input}")

messages = [system, MessagesPlaceholder(variable_name='history'), human]

prompt = ChatPromptTemplate(messages=messages)

chain = prompt | llm | StrOutputParser()

runnable_with_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key='input', history_messages_key='history')

In [24]:
def chat_with_llm(session_id, input):
    output = runnable_with_history.invoke({'input':input}, config={'configurable':{'session_id':session_id}})
    return output

In [27]:
user_id = "son"
chat_with_llm(user_id, "My name is Sungjong Son. I work for DevCom")

"You work in DevCom, which means you're likely involved in the development and commissioning of computer systems and software. Is there anything on your mind that you'd like to discuss or ask about in relation to your role at DevCom?"

In [28]:
chat_with_llm(user_id, "What is my name?")

'I remember! Your name is Sungjong Son, and you work for DevCom.'